# Creation of Animated Star Map

Pretty much the same with the `map_create.ipynb` except that this notebook would generate multiple instances at different times and then convert it into a short video

## Importing Packages

In [41]:
from astropy.coordinates import EarthLocation, AltAz, get_sun, get_body, SkyCoord
from astropy.time import Time
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import pandas as pd
import ast
from pathlib import Path
import os
from moviepy.editor import ImageSequenceClip
from moviepy.editor import VideoFileClip
from matplotlib.backends.backend_agg import FigureCanvasAgg

## Setting up directories

In [42]:
working_dir = Path.cwd()
os.chdir(working_dir)

with open("Resources Directory.txt", "r") as resources_text:
    resources_dir = Path(resources_text.readline())

workspace_dir = resources_dir / "Video Related Files"

if not workspace_dir.exists():
    os.mkdir(workspace_dir)

data_process_dir = working_dir / "Data Processed"
connection_dir = data_process_dir / "Constellation Connection"
clean_star_list_dir = data_process_dir / "Clean_Star_List.csv"

## Reading the stars

In [43]:
clean_frame = pd.read_csv(clean_star_list_dir)

star_list = []
for index, row in clean_frame.iterrows():
    star_RA = row["RA"]
    star_Dec = row["Dec"]
    star_VMag = row["Vmag"]
    star_PMag = row["Pmag"]
    star_alpha = row["AlphaPlot"]
    value_list = [star_RA, star_Dec, star_VMag, star_PMag, star_alpha]

    star_list.append(value_list)

## Reading the connections

In [44]:
outer_connection_list = []
for connection_file in connection_dir.iterdir():
    connection_frame = pd.read_csv(connection_file)

    connection_list = []

    for index, row in connection_frame.iterrows():
        RA_points = np.array(ast.literal_eval(row["RA Points"]))
        Dec_points = np.array(ast.literal_eval(row["Dec Points"]))

        combination_pack = np.array([RA_points, Dec_points])

        connection_list.append(combination_pack)

    connection_count = 0
    outer_connection_list.append(connection_list)

## Processing stars

In [45]:
count = 0
ra_list = []
dec_list = []
size_list = []
alpha_list = []

star_amount = 1000

while count < star_amount:
    current_star = star_list[count]
    ra_list.append(current_star[0])
    dec_list.append(current_star[1])
    size_list.append(current_star[3])
    alpha_list.append(current_star[4])
    count += 1

np_ra_list = np.array(ra_list)
np_dec_list = np.array(dec_list)

## Defining some functions

In [46]:
def altaz_to_stereographic(alt, az):

    # Converting altitude and azimuth to radian
    # Altitude is taken as negative since projection comes from the south pole
    alt_rad = -alt.to(u.rad)
    az_rad = az.to(u.rad)

    # Convert altitude and azimuth to Cartesian coordinates
    x = np.cos(alt_rad) * np.sin(az_rad)
    y = np.cos(alt_rad) * np.cos(az_rad)
    z = np.sin(alt_rad)

    # Project onto the 2D plane (stereographic projection)
    rho = 1 / (1 - z)
    x_proj = rho * x
    y_proj = rho * y

    return x_proj, y_proj


def altaz_to_stereographic_calculate(alt, az):

    # Convert altitude and azimuth to Cartesian coordinates
    x = np.cos(np.radians(-alt)) * np.sin(np.radians(az))
    y = np.cos(np.radians(-alt)) * np.cos(np.radians(az))
    z = np.sin(np.radians(-alt))

    # Project onto the 2D plane (stereographic projection)
    rho = 1 / (1 - z)
    x_proj = rho * x
    y_proj = rho * y

    return x_proj, y_proj

## Setting up static objects

### Here the equator and ecliptic are defined

In [47]:
longitude = np.linspace(0, 360, 360+1) * u.deg
latitude = 0 * u.deg

# Equator is computed here
equator_coords = SkyCoord(ra=longitude, dec=latitude, unit='deg', frame='icrs')

# Ecliptic is computed here
ecliptic_coords = SkyCoord(lon=longitude, lat=latitude,
                           unit='deg', frame='barycentrictrueecliptic')

### Here the points that designate altitude and other important points are defined

In [48]:
azimuth_points = np.linspace(0, 360, 1440+1)

x_proj_0, y_proj_0 = altaz_to_stereographic_calculate(0, azimuth_points)
x_proj_15, y_proj_15 = altaz_to_stereographic_calculate(15, azimuth_points)
x_proj_30, y_proj_30 = altaz_to_stereographic_calculate(30, azimuth_points)
x_proj_45, y_proj_45 = altaz_to_stereographic_calculate(45, azimuth_points)
x_proj_60, y_proj_60 = altaz_to_stereographic_calculate(60, azimuth_points)
x_proj_75, y_proj_75 = altaz_to_stereographic_calculate(75, azimuth_points)

# Semicircle points
x_semicircle = np.linspace(-1, 1, 512+1)
y_circle_above = np.sqrt(1-(x_semicircle**2))
y_circle_below = -np.sqrt(1-(x_semicircle**2))

# Outer ring points
angle_position = np.linspace(0, 2*np.pi, 1440+1)
x_outer_ring = np.cos(angle_position)
y_outer_ring = np.sin(angle_position)

In [49]:
count = 0

top_semicircle_vertices = [[-1.25, 1.25], [-1.25, 0]]
bottom_semicircle_vertices = [[-1.25, -1.25], [-1.25, 0]]

while count < len(x_semicircle):

    top_list = [x_semicircle[count], y_circle_above[count]]
    bot_list = [x_semicircle[count], y_circle_below[count]]

    top_semicircle_vertices.append(top_list)
    bottom_semicircle_vertices.append(bot_list)

    count += 1

top_semicircle_vertices += [[1.25, 0], [1.25, 1.25]]
bottom_semicircle_vertices += [[1.25, 0], [1.25, -1.25]]

top_array = np.array(top_semicircle_vertices)
bottom_array = np.array(bottom_semicircle_vertices)

square_vertices = np.array(
    [[1.25, 1.25], [-1.25, 1.25], [-1.25, -1.25], [1.25, -1.25]])

### Creating background mask

In [50]:
fig, ax = plt.subplots()

fig.set_size_inches(8, 8)

dpi = 135  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

ax.fill(square_vertices[:, 0], square_vertices[:, 1], color='#FFFFFF', alpha=1)
ax.fill(top_array[:, 0], top_array[:, 1], color=(0, 0, 0), alpha=1)
ax.fill(bottom_array[:, 0], bottom_array[:, 1], color=(0, 0, 0), alpha=1)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Background_Mask.png"
PIL_image.save(output_path)
plt.close(fig)

### Creating the actual background image

In [51]:
background_mask_img_path = workspace_dir / "Background_Mask.png"
out_background_mask_img_path = workspace_dir / "Background_01.jpg"

background_mask_img = Image.open(str(background_mask_img_path)).convert("L")
background_mask_img = ImageOps.invert(background_mask_img)

out_background_mask_img = Image.open(
    str(out_background_mask_img_path)).convert("RGBA")
out_background_mask_img = out_background_mask_img.resize((background_mask_img.size[0], background_mask_img.size[1]), Image.LANCZOS)

result_image = Image.new("RGBA", out_background_mask_img.size, (0, 0, 0, 0))
result_image.paste(out_background_mask_img, mask=background_mask_img)

output_path = workspace_dir / "Background_Image.png"
result_image.save(str(output_path))

## Creating the static images

### Rim

In [52]:
fig, ax = plt.subplots()

fig.set_size_inches(8, 8)

dpi = 135  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

ax.plot(1.08*x_outer_ring, 1.08*y_outer_ring, color=(1, 1, 1),linewidth=0.5)
ax.scatter(1.05*x_outer_ring[::45], 1.05*y_outer_ring[::45], color=(1, 1, 1),s=1)
ax.plot(1.02*x_outer_ring, 1.02*y_outer_ring, color=(1, 1, 1),linewidth=0.5)
ax.plot(x_proj_0, y_proj_0, color=(1, 1, 1), linewidth=2.5)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Rim.png"
PIL_image.save(output_path)
plt.close(fig)

### Rings

In [53]:
fig, ax = plt.subplots()

fig.set_size_inches(8, 8)

dpi = 135  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

ax.scatter(x_proj_15[::90][:-1], y_proj_15[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_30[::90][:-1], y_proj_30[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_45[::90][:-1], y_proj_45[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_60[::90][:-1], y_proj_60[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_75[::90][:-1], y_proj_75[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(0, 0, color=(0.25, 0.5, 1), s=0.5, alpha=0.25)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Rings.png"
PIL_image.save(output_path)
plt.close(fig)

## Setting up the static images directory

In [54]:
background_img_path = workspace_dir / "Background_01.jpg"
rings_img_path = workspace_dir / "Rings.png"
rim_img_path = workspace_dir / "Rim.png"
out_background_img_path = workspace_dir / "Background_Image.png"

## Loading up static images

In [55]:
background_img = Image.open(str(background_img_path))
background_img = background_img.resize((1080, 1080), Image.LANCZOS)

rings_img = Image.open(str(rings_img_path))
rim_img = Image.open(str(rim_img_path))
out_background_img = Image.open(str(out_background_img_path))

background_img = background_img.convert('RGBA')
rings_img = rings_img.convert('RGBA')
rim_img = rim_img.convert('RGBA')
out_background_img = out_background_img.convert('RGBA')

# Non Static Parts

First set up observer location

In [56]:
latitude = 0
longitude = 0

observer_location = EarthLocation(lat=latitude, lon=longitude)

Now set up the starting time

In [57]:
date_time = Time.now()
plot_time = date_time

iso_time = "2000-01-01T00:00:00"  # YYYY-MM-DDTHH:MM:SS
specific_time = Time(iso_time, format='isot', scale='utc')
plot_time = specific_time

Defining some functions to make the loop plot cleaner

In [58]:
def acquire_altaz_frame(observer_location):
    # AltAz frame is set here
    AltAz_frame = AltAz(location=observer_location)
    return AltAz_frame

def acquire_celestial_paths(observer_location, plot_time):
    # Equator and ecliptic converted to AltAz
    equator_coord_altaz = equator_coords.transform_to(AltAz(location=observer_location, obstime=plot_time))
    ecliptic_coord_altaz = ecliptic_coords.transform_to(AltAz(location=observer_location, obstime=plot_time))
    return equator_coord_altaz, ecliptic_coord_altaz

def acquire_solar_objects(AltAz_frame,plot_time):
    # Getting the position of the sun, moon, and the eight planets
    sun_position = get_sun(plot_time).transform_to(AltAz_frame)
    moon_position = get_body("moon", plot_time).transform_to(AltAz_frame)
    mercury_position = get_body("mercury", plot_time).transform_to(AltAz_frame)
    venus_position = get_body("venus", plot_time).transform_to(AltAz_frame)
    mars_position = get_body("mars", plot_time).transform_to(AltAz_frame)
    jupiter_position = get_body("jupiter", plot_time).transform_to(AltAz_frame)
    saturn_position = get_body("saturn", plot_time).transform_to(AltAz_frame)
    uranus_position = get_body("uranus", plot_time).transform_to(AltAz_frame)
    neptune_position = get_body("neptune", plot_time).transform_to(AltAz_frame)
    return sun_position,moon_position,mercury_position,venus_position,mars_position,jupiter_position,saturn_position,uranus_position,neptune_position

def acquire_solar_paths(AltAz_frame, plot_time):
    # Creating time entries for future
    year_forward = plot_time + np.linspace(0, 365, 365+1)*24*u.hour
    day_forward = plot_time + np.linspace(0, 24, 96+1)*u.hour

    # Acquiring position of the analemma and the path of the sun
    analemma_position = get_sun(year_forward).transform_to(AltAz_frame)
    day_path_position = get_sun(day_forward).transform_to(AltAz_frame)

    return analemma_position, day_path_position

# Loop to plot at different times

Change the `time_step_amount` and the `time_step_size` to desired values

In [59]:
new_frame_list = []

AltAz_frame = acquire_altaz_frame(observer_location)

time_step_amount = 240
time_step_size = (24/120)*u.hour

time_step_count = 0

while time_step_count < time_step_amount:
    print(f"{time_step_count+1}/{time_step_amount} | {plot_time}")

    equator_coord_altaz, ecliptic_coord_altaz = acquire_celestial_paths(observer_location, plot_time)
    analemma_position, day_path_position = acquire_solar_paths(AltAz_frame, plot_time)
    sun_position,moon_position,mercury_position,venus_position,mars_position,jupiter_position,saturn_position,uranus_position,neptune_position = acquire_solar_objects(AltAz_frame,plot_time)

    # For the sun, moon, and the eight planets
    x_sun, y_sun = altaz_to_stereographic(sun_position.alt, sun_position.az)
    x_moon, y_moon = altaz_to_stereographic(moon_position.alt, moon_position.az)
    x_mercury, y_mercury = altaz_to_stereographic(mercury_position.alt, mercury_position.az)
    x_venus, y_venus = altaz_to_stereographic(venus_position.alt, venus_position.az)
    x_mars, y_mars = altaz_to_stereographic(mars_position.alt, mars_position.az)
    x_jupiter, y_jupiter = altaz_to_stereographic(jupiter_position.alt, jupiter_position.az)
    x_saturn, y_saturn = altaz_to_stereographic(saturn_position.alt, saturn_position.az)
    x_uranus, y_uranus = altaz_to_stereographic(uranus_position.alt, uranus_position.az)
    x_neptune, y_neptune = altaz_to_stereographic(neptune_position.alt, neptune_position.az)

    # For the equator and ecliptic
    equator_x, equator_y = altaz_to_stereographic(equator_coord_altaz.alt, equator_coord_altaz.az)
    ecliptic_x, ecliptic_y = altaz_to_stereographic(ecliptic_coord_altaz.alt, ecliptic_coord_altaz.az)

    # For the analemma and path of the sun
    analemma_x, analemma_y = altaz_to_stereographic(analemma_position.alt, analemma_position.az)
    day_path_x, day_path_y = altaz_to_stereographic(day_path_position.alt, day_path_position.az)

    fig, ax = plt.subplots()

    fig.set_size_inches(8, 8)

    dpi = 135  # Set your desired DPI value
    fig.set_dpi(dpi)
    fig.patch.set_alpha(0)

    
    ax.plot(equator_x, equator_y, linestyle='--', dashes=(30, 20),color=(0.75, 0.5, 1), linewidth=0.25)
    ax.plot(ecliptic_x, ecliptic_y, linestyle='--',dashes=(30, 20), color=(0.25, 1, 0.5), linewidth=0.25)

    ax.plot(analemma_x, analemma_y, linestyle='--', dashes=(15, 10),color=(0.5, 0.25, 0.125), linewidth=0.25)
    ax.plot(day_path_x, day_path_y, linestyle='--',dashes=(30, 20), color=(1, 0.5, 0.25), linewidth=0.25)


    for outer_connection in outer_connection_list:
        for connection in outer_connection:
            line_coords = SkyCoord(ra=connection[0], dec=connection[1], unit='deg', frame='icrs')
            line_altaz = line_coords.transform_to(AltAz(location=observer_location, obstime=plot_time))

            line_x, line_y = altaz_to_stereographic(line_altaz.alt, line_altaz.az)
            ax.plot(line_x, line_y, color=(1, 1, 1), linewidth=0.5, alpha=0.75)

    star_coords = SkyCoord(ra=np_ra_list, dec=np_dec_list, unit='deg', frame='icrs')
    star_altaz = star_coords.transform_to(AltAz(location=observer_location, obstime=plot_time))

    star_x, star_y = altaz_to_stereographic(star_altaz.alt, star_altaz.az)
    ax.scatter(star_x, star_y, marker=".", color=(1, 1, 1), s=size_list, alpha=alpha_list)

    ax.scatter(x_neptune, y_neptune, label="Neptune", marker=r"$♆$",color=(0.75, 0, 1))
    ax.scatter(x_uranus, y_uranus, label="Uranus", marker=r"$⛢$",color=(0.678, 0.847, 0.902))
    ax.scatter(x_saturn, y_saturn, label="Saturn", marker=r"$♄$",color=(0.647, 0.165, 0.165))
    ax.scatter(x_jupiter, y_jupiter, label="Jupiter", marker=r"$♃$",color=(0, 0, 1))
    ax.scatter(x_mars, y_mars, label="Mars", marker=r"$♂$",color=(1, 0, 0))
    ax.scatter(x_venus, y_venus, label="Venus", marker=r"$♀$",color=(0, 1, 0))
    ax.scatter(x_mercury, y_mercury, marker=r'$☿$',color=(1, 1, 0))

    ax.scatter(x_sun, y_sun, label="Sun", color=(1, 0.843, 0))
    ax.scatter(x_moon, y_moon, label="Moon", color=(0.5, 0.5, 0.5), edgecolors=(1, 1, 1))


    # Set limits
    ax.set_xlim(-1.25, 1.25)
    ax.set_ylim(-1.25, 1.25)
    ax.grid(False)
    # Remove axes and labels
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Remove frame
    ax.set_frame_on(False)

    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    canvas = FigureCanvasAgg(fig)

    # Render the figure into a numpy array
    canvas.draw()

    image_array = np.array(canvas.buffer_rgba())
    
    mobile_plot = Image.fromarray(image_array)

    overlay = Image.alpha_composite(background_img, rings_img)
    overlay = Image.alpha_composite(overlay, mobile_plot)
    overlay = Image.alpha_composite(overlay, out_background_img)
    overlay = Image.alpha_composite(overlay, rim_img)
    new_frame_list.append(overlay.convert('RGB'))
    plt.close(fig)

    plot_time = plot_time + time_step_size
    time_step_count += 1

1/240 | 2000-01-01T00:00:00.000
2/240 | 2000-01-01T00:12:00.000
3/240 | 2000-01-01T00:24:00.000
4/240 | 2000-01-01T00:36:00.000
5/240 | 2000-01-01T00:48:00.000
6/240 | 2000-01-01T01:00:00.000
7/240 | 2000-01-01T01:12:00.000
8/240 | 2000-01-01T01:24:00.000
9/240 | 2000-01-01T01:36:00.000
10/240 | 2000-01-01T01:48:00.000
11/240 | 2000-01-01T02:00:00.000
12/240 | 2000-01-01T02:12:00.000
13/240 | 2000-01-01T02:24:00.000
14/240 | 2000-01-01T02:36:00.000
15/240 | 2000-01-01T02:48:00.000
16/240 | 2000-01-01T03:00:00.000
17/240 | 2000-01-01T03:12:00.000
18/240 | 2000-01-01T03:24:00.000
19/240 | 2000-01-01T03:36:00.000
20/240 | 2000-01-01T03:48:00.000
21/240 | 2000-01-01T04:00:00.000
22/240 | 2000-01-01T04:12:00.000
23/240 | 2000-01-01T04:24:00.000
24/240 | 2000-01-01T04:36:00.000
25/240 | 2000-01-01T04:48:00.000
26/240 | 2000-01-01T05:00:00.000
27/240 | 2000-01-01T05:12:00.000
28/240 | 2000-01-01T05:24:00.000
29/240 | 2000-01-01T05:36:00.000
30/240 | 2000-01-01T05:48:00.000
31/240 | 2000-01-01

Storing the frames as numpy arrays

In [60]:
np_frame_list = []
for frame in new_frame_list:
    np_frame_list.append(np.array(frame))

Creation of video

In [ ]:
fps=60

new_video = ImageSequenceClip(np_frame_list, fps=fps)
output_path = resources_dir / f"Output Video.mp4"
new_video.write_videofile(str(output_path), codec='libx264')